<a href="https://colab.research.google.com/github/oiChuva/AI-A3-2Semetre2024/blob/main/codeforAIusjt2Sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install uvicorn
!pip install fastapi
!pip install python-multipart

In [ ]:
from fastapi import FastAPI, HTTPException, Request
import cv2
from google.colab.patches import cv2_imshow
import uvicorn
import os.path
from os import path
import numpy as np
from fastapi.responses import JSONResponse
from fastapi.responses import HTMLResponse

In [ ]:
app = FastAPI.FastAPI()

if path.exists('/content/NoProcessing') == False:
  os.mkdir('/content/NoProcessing')

if path.exists('/content/Processed') == False:
  os.mkdir('/content/Processed')

A ideia da api é ela ser um post, teremos uma função principal que utiliza
a imagem recebida na IA, que, conforme a API receber o conjunto de dados da
validação, imagem, tipo de item e pergunta(tem ou não tem?), jogará a
imagem na IA e manda uma response para o usuário.

# Este código é para codar em colaboração, o código rodará no VSC.

In [ ]:
def image_treat():
  for filename in sorted(os.listdir('/content/NoProcessing')):
    if filename.endswith(".png"):  #Precisa ser PNG. (Para evitar problemas melhor setar só pra um)
        image_path = os.path.join('/content/NoProcessing', filename)
        imagem = cv2.imread(image_path)

        if imagem is not None:
            (height, width, depth) = imagem.shape

            # - Escala de CINZA AQUI -------------------------------------------------------------
            imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

            # - Suavização da IMAGEM AQUI
            imagemSuavizada = cv2.GaussianBlur(imagem, (3,3), 0)

            # - CONTORNOS AQUI
            imagemContornadaX = cv2.Sobel(imagemSuavizada, cv2.CV_64F, 1, 0)
            imagemContornadaX = np.uint8(np.absolute(imagemContornadaX))
            imagemContornadaY = cv2.Sobel(imagemSuavizada, cv2.CV_64F, 0, 1)
            imagemContornadaY = np.uint8(np.absolute(imagemContornadaY))
            imagemContornadaXY = cv2.bitwise_or(imagemContornadaX, imagemContornadaY)

            # Só para ter certeza que esta funcionando - PARA VER SE FUNCIONA, DESCOMENTE A PRÓXIMA LINHA
            # cv2_imshow(imagemContornadaXY)

            output_path = os.path.join('/content/Processed', f"Processed_{filename}")
            cv2.imwrite(output_path, imagemContornadaXY)
    else:
            print(f"Error ao carregar {filename}")

In [ ]:
#código da IA vai aqui
def IA(imagem, tipo_p):
  return

In [ ]:
@app.post("/AI-Pensa")
async def webhook(request: Request):
    """Recebe dados via webhook e processa a inclusão."""
    try:
        data = await request.json()
        print(f"Received data: {data}")
        imagem = data.get("foto")
        if imagem is None:
            raise HTTPException(status_code=400, detail="Foto não fornecida.")

        tipo_p = data.get("tipo")
        if tipo_p is None:
            raise HTTPException(status_code=400, detail="Tipo não fornecido.")

        # processa imagem
        imagem_processada = image_treat(imagem)
        if "error" in imagem_processada:
            raise HTTPException(status_code=400, detail=imagem_processada["error"])

        #dá a imagem processada para a IA resolver se tem o item ou não
        resultado = IA(imagem_processada, tipo_p)
        if "error" in resultado:
            raise HTTPException(status_code=400, detail=resultado["error"])

        return JSONResponse(content={resultado})
        # tipo é o que a IA vai analisar da foto, pessoa, faca, cadeira, lixeira, por exemplo. _p de pergunta
        # foto é o que vem da requisição, imagem é o que vai ser tratado e a IA vai analisar.

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/test-page", response_class=HTMLResponse)
async def test_page():
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Server Test Page</title>
        <style>
            body { font-family: Arial, sans-serif; background-color: #f4f4f4; color: #333; text-align: center; padding: 50px; }
            h1 { color: #4CAF50; }
            p { font-size: 1.2em; }
            .status { font-weight: bold; }
        </style>
    </head>
    <body>
        <h1>Server is Running!</h1>
        <p>Welcome to the <span class="status">FastAPI Server Test Page</span></p>
        <p>Status: <span class="status">OK</span></p>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)
